In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(''), '..'))

import numpy
import pandas
import custom
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from IPython.display import display


In [2]:
def word_vectorize(sentence : [str,list], vec_dict : dict, word_len : [int, None] = None) :
    temp = []
    
    if type(sentence) == str : 
        words = str(sentence).split()
    else :
        words = sentence
    if word_len is None :
        word_len = len(sentence)
        
    for i in range(word_len - len(words)) :
        temp.append(vec_dict["<pad>"])
    for i in range(len(words)) :
        if words[i] not in vec_dict :
            temp.append(vec_dict["<unk>"])
            continue
        temp.append(vec_dict[words[i]])

    return temp

In [3]:
##데이터 불러오기
df = pandas.read_csv("data/simple_movie_data.csv", encoding="UTF8")
display(df)

with open("data/vector.pkl", mode="rb") as f:
    vec = pickle.load(f)

print(len(vec))

,quote,scoreSentiment,word_len
0,if you have not seen it do,1,7
1,it s benchmark cinema a highpoint of movie his...,1,9
2,this movie is all about brando,1,6
3,this is sean connery at his best,1,7
4,there are so many questions that the script ne...,0,10
...,...,...,...
101047,technically a little bit crude but it delivers,1,8
101048,a sturdy N entertaining late night watch N,1,8
101049,what on earth happened here N,0,6
101050,is it tuesday yet,0,4


124833


In [4]:
##데이터 전처리

sentences = df["quote"].values.tolist()
t = df["scoreSentiment"].values.tolist()
word_len = df["word_len"].max()

x = []
for s in sentences :
    s = custom.text_preprocess(s)
    x.append(word_vectorize(s, vec, word_len))
x = numpy.array(x)

print(x.shape)
    

(101052, 10, 300)


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

##DataLoader 생성
tensor_x = torch.tensor(x, dtype = torch.float, device = device)
tensor_t = torch.tensor(t, dtype = torch.long, device = device)
zip_list = list(zip(tensor_x, tensor_t))
dataloader = DataLoader(zip_list, batch_size= 200, shuffle= True)
test_dataloader = DataLoader(zip_list, batch_size= 10000, shuffle= False)

print(len(dataloader))
print(len(test_dataloader))



506
11


In [6]:
##AI 만들기
#함수 선언
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.rnn = nn.LSTM(300,10,batch_first = True,device = device)
        self.f = nn.Sequential(
            nn.Linear(10,2),
            nn.ReLU()
        )
    def forward(self, x) :
        x, h = self.rnn(x)
        x = x[:,-1,:]
        x = self.f(x)
        return x
F = NN()


In [7]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(F.parameters(), lr = 2)
epoch = 30

for e in range(epoch) :
    loss_sum = 0
    for x, t in dataloader :
    # x, t 입력
    # y = F(x)
        y = F(x)
    # 손실함수
        loss = loss_function(y,t)
        loss_sum += loss
    # 최적화함수
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"epoch {e+1} || loss {loss_sum / len(dataloader)}")

epoch 1 || loss 0.5452408790588379
epoch 2 || loss 0.4808312654495239
epoch 3 || loss 0.468647837638855
epoch 4 || loss 0.4610574245452881
epoch 5 || loss 0.45565298199653625
epoch 6 || loss 0.4094552993774414
epoch 7 || loss 0.38706666231155396
epoch 8 || loss 0.37621432542800903
epoch 9 || loss 0.3655543029308319
epoch 10 || loss 0.3589748740196228
epoch 11 || loss 0.3520294427871704
epoch 12 || loss 0.34668344259262085
epoch 13 || loss 0.3393911123275757
epoch 14 || loss 0.3346388041973114
epoch 15 || loss 0.33082181215286255
epoch 16 || loss 0.32687440514564514
epoch 17 || loss 0.3223250210285187
epoch 18 || loss 0.3175757825374603
epoch 19 || loss 0.3137519657611847
epoch 20 || loss 0.31128910183906555
epoch 21 || loss 0.3072192966938019
epoch 22 || loss 0.30466362833976746
epoch 23 || loss 0.30182793736457825
epoch 24 || loss 0.29864034056663513
epoch 25 || loss 0.2958507239818573
epoch 26 || loss 0.2933337092399597
epoch 27 || loss 0.29012730717658997
epoch 28 || loss 0.28862592

In [8]:
cnt = 0
total = 0

for x, t in test_dataloader :
    y = F(x)
    cnt += (y.argmax(dim=1) == t).sum().item()
    total += len(x)

print("possiblity : %f" %(cnt / total))

possiblity : 0.874659


In [9]:
torch.save(F.to("cpu"), "simple_movie_LSTM.pt")